<b>Der IMBD-Datensatz</b><br>
- 50 000 Reviews der Internet Movie Database, 25 000 gut, alle weiteren schlecht
- es ist bereits in Keras vorhanden

In [10]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 100)
# num_words: how many of the most commonly used words included (so that vectors are managebly small)
print(len(test_labels))
print(len(train_data))
print(len(train_data[0]))

25000
25000
218


Warum wurden die Daten in Trainingsdaten und Testdaten geteilt? Weil ein Modell nicht auf den gleichen Daten trainiert und getestet werden sollte. Das liegt daran, dass "Overfitting" verhindert werden soll. Das bedeutet, dass das Modell die Daten auswendig lernt, das Modell also so spezifisch auf die Trainigsdaten gefittet ist, dass es keine auf weitere Daten übertragbare Abstraktion ist. Das Modell ist also zu sehr auf die Eigenheiten des vorliegenden Datensatzes angepasst. Um zu verhindern, dass die Loss Function anhand auswendig gerlernter Daten ermittelt wird, sondern die Leistungsfähigkeit des Modells auf unbekannte Daten, benutzt man getrennte Test- und Trainigsdaten.

In [2]:
print(train_data[0])
print(train_labels[0])

[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 4, 2, 36, 2, 5, 25, 2, 43, 2, 2, 50, 2, 2, 9, 35, 2, 2, 5, 2, 4, 2, 2, 2, 2, 2, 2, 39, 4, 2, 2, 2, 17, 2, 38, 13, 2, 4, 2, 50, 16, 6, 2, 2, 19, 14, 22, 4, 2, 2, 2, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 2, 12, 8, 2, 8, 2, 5, 4, 2, 2, 16, 2, 66, 2, 33, 4, 2, 12, 16, 38, 2, 5, 25, 2, 51, 36, 2, 48, 25, 2, 33, 6, 22, 12, 2, 28, 77, 52, 5, 14, 2, 16, 82, 2, 8, 4, 2, 2, 2, 15, 2, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 7, 4, 2, 2, 13, 2, 88, 4, 2, 15, 2, 98, 32, 2, 56, 26, 2, 6, 2, 2, 18, 4, 2, 22, 21, 2, 2, 26, 2, 5, 2, 30, 2, 18, 51, 36, 28, 2, 92, 25, 2, 4, 2, 65, 16, 38, 2, 88, 12, 16, 2, 5, 16, 2, 2, 2, 32, 15, 16, 2, 19, 2, 32]
1


Die Daten bestehen aus Sequenzen von Worten, die als Zahlen codiert worden sind (0 bis 9999). Die Labels, also die Ziele sind entweder 1 oder 0. 1 steht für eine gute Review und 0 für eine schlechte.

<b>Vorbereitung der Daten</b><br>
Es ist einfach möglich, Listen unterschiedlicher Länge von Integeres in ein neuronales Netz zu leiten.
- die Inputvektoren müssen eine einheitliche Länge haben,
- das kann mit One-Hot-Kodierung erreicht werden
- so wird die Sequenz <code>[3, 5]</code> in einen Vektor der Länge 10 000 umgewandelt, der an den Stellen 3 und Einsen hätte und sonst Nullen

In [4]:
import numpy as np

def vectorizeData(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorizeData(train_data)
x_test = vectorizeData(test_data)

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

In [7]:
print(x_train[0])
print(x_test[0])
print(y_train[0])
print(y_test[0])

[0. 1. 1. ... 0. 0. 0.]
[0. 1. 1. ... 0. 0. 0.]
1.0
0.0


Der letzte Schritt bezüglich der Labels ist ebenfalls wichtig. Keras erwartet NumPy-Arrays vom Typ float32.

<b>Der Bau des Netzwerks</b><br>


In [11]:
import keras
layer1 = Dense(16, activation = 'relu')

NameError: name 'Dense' is not defined

Ein Setup mit Vektoren als Input und einem binären Output ist das einfachste, dass einem begegnen kann. Diese werden am besten durch vollständig vernetzte Schichten <code>Dense</code> verarbeitet.<br>
Sei <code>W</code> die Matrix der Gewichte der Schicht mit den Dimensionen (inputs, 16), also jede Spalte/jedes Neuron/jede "versteckte Einheit" ordnet jedem Input/jeder Zeile ein Gewicht zu.<br>
<code>output = relu(dot(W, input) + bias)</code><br>
Der Output einer Schicht wird also ermittelt, indem die Inputs über <code>W</code> gewichtet werden. Nun hat man einen Vektor der Länge 16, bei dem jedes Element die individuell gewichtete Summe der Inputs ist. Dazu wird ein Bias-Vektor addiert und auf dieses Ergebnis die ReLU-Funktion angewandt. Mehr dazu später.<br>
Die 16 ist die Zahl versteckter Einheiten der Schicht. Diese Zahl lässt sich als der mögliche Repräsentationsraum verstehen. Je mehr versteckte Einheiten, desto komplexer sind die Muster, die gelernt werden können. Allerdings auch desto rechnerisch teuerer und anfälliger für Overfitting ist das Netzwerk.<br><br>
Es gibt nun zwei wichige Fragen, wenn es zu den <code>Dense</code>-Schichten kommt:
1. Wie viele Schichten sollten genutzt werden,
2. wie viele versteckte Enheiten sollte die jeweilige Schicht besitzen?<br>
Formale Prinzipien werden später erläutert. 

Für's Erste werden wir